# Using sktime models

This is an example for using sktime based models with the `timecopilot` library.

## imports

In [1]:
import nest_asyncio
nest_asyncio.apply()

import timecopilot
import sktime as skt
import pandas as pd

## Setup the sktime model and adapt it to TimeCopilot

sktime models need to be adapted to work properly with TimeCopilot. This is done by creating your model with sktime and passing it through SKTimeAdapter. Some sktime models may require more configuration to function properly with the data you intend to use it on. For example, when using sktime's NaiveForecaster with yearly data you might want to initialize it with an `sp` argument of `12` like this `NaiveForecaster(sp=12)`.

The `Alias` argument should also be provided, especially if you plan on adding multiple sktime forecasters. If you add multiple sktime models without specifying aliases, TimeCopilot will not be able to properly call all of them.

In [ ]:
from sktime.forecasting.trend import TrendForecaster
from timecopilot.models.adapters.sktime import SKTimeAdapter

trend_forecaster = TrendForecaster()

adapted_model = SKTimeAdapter(
    model=trend_forecaster,
    alias="TrendForecaster",
)

## Create a TimeCopilot instance with your sktime model

You will need to specify the forecasters you're using when using sktime models. 

In [3]:
tc = timecopilot.TimeCopilot(
    llm="openai:gpt-4o",
    forecasters=[
        adapted_model,
    ],
)

### Extending default model list with an sktime adapted model

if you want to use the default list with the addition of your sktime model you could make a copy of the default list and append your model to it:

In [ ]:
model_list = timecopilot.agent.DEFAULT_MODELS.copy()
model_list.append(adapted_model)

tc = timecopilot.TimeCopilot(
    llm="openai:gpt-4o",
    forecasters=model_list
)

## Forecasting 
Once that setup is complete, you can use TimeCopilot with your adapted sktime model the same way you'd normally use TimeCopilot

In [4]:
df = pd.read_csv("https://timecopilot.s3.amazonaws.com/public/data/air_passengers.csv")

In [5]:
result = tc.forecast(
    df=df,
)

1it [00:00,  4.70it/s]
1it [00:00, 223.32it/s]
11it [00:00, 77.11it/s]


In [10]:
print(result.output.tsfeatures_analysis)

The 'AirPassengers' time series has a series length of 144 with a clear seasonal pattern identified using key features. The high 'seasonal_strength' of 0.981 suggests strong seasonality, evident from the 12-month seasonal period. The time series also exhibits trends, shown by a 'trend' score of 0.997, and moderate curvature at 1.069. The high autocorrelation 'x_acf1' at 0.948 indicates the persistence of patterns over time. The Holt-Winters parameters suggest a stable level (alpha ~1) with no trend component (beta ~0) and significant seasonal smoothing (gamma ~0.75). These features suggest that both trend and seasonality are prominent and need to be captured by the model.


In [8]:
result.fcst_df

,unique_id,ds,TrendForecaster
0,AirPassengers,1961-01-01,473.023018
1,AirPassengers,1961-02-01,475.729097
2,AirPassengers,1961-03-01,478.173296
3,AirPassengers,1961-04-01,480.879374
4,AirPassengers,1961-05-01,483.498159
5,AirPassengers,1961-06-01,486.204237
6,AirPassengers,1961-07-01,488.823023
7,AirPassengers,1961-08-01,491.529101
8,AirPassengers,1961-09-01,494.235179
9,AirPassengers,1961-10-01,496.853964
